In [ ]:
import os
import glob
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.layers import TimeDistributed 
import tkinter as tk
from tkinter import filedialog
from pydub import AudioSegment
import sounddevice as sd
from PIL import Image, ImageTk 

In [ ]:
def extract_features(file_path):
    audio, sr = librosa.load(file_path, res_type='kaiser_fast')
    features = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T, axis=0)
    return features


In [ ]:
data = []
labels = []
for folder_name in os.listdir('archive\TESS Toronto emotional speech set data\datasets'):
    folder_path = os.path.join('archive\TESS Toronto emotional speech set data\datasets', folder_name)
    for file_path in glob.glob(os.path.join(folder_path, '*.wav')):
        print(file_path)
        features = extract_features(file_path)
        data.append(features)
        labels.append(folder_name)

In [ ]:
import pandas as pd

df = pd.DataFrame(data)
df['label'] = labels

df.to_csv('extracted_features_testDatasets.csv', index=False)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

X_train = np.array(X_train)[:, np.newaxis, :]
X_test = np.array(X_test)[:, np.newaxis, :]

model = Sequential()
model.add(TimeDistributed(Dense(256, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=75, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Extract training and validation accuracy and loss from history
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot accuracy
plt.figure(figsize=(6, 4))
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.savefig('accuracy_plot.png')  # Simpan gambar akurasi

# Plot and save loss
plt.figure(figsize=(6, 4))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.savefig('loss_plot.png') 
plt.show()

loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
model.save("model_terbaik.h5");

In [ ]:

def predict_emotion(audio_file):
    best_model = tf.keras.models.load_model('model_terbaik1.h5')
    features = extract_features(audio_file)
    features = features[np.newaxis, np.newaxis, :]  
    print("Features shape:", features.shape)
    print("Features:", features)

    predicted_probabilities = best_model.predict(features)
    print("Predicted probabilities shape:", predicted_probabilities.shape)
    print("Predicted probabilities:", predicted_probabilities)

    predicted_label_index = np.argmax(predicted_probabilities)
    print("Predicted label index:", predicted_label_index)

    predicted_emotion = label_encoder.classes_[predicted_label_index]
    print("Predicted emotion:", predicted_emotion)


    # Emotion mapping for TESS dataset
    emotion_mapping = {
        'angry': 'ANGRY',
        'disgust': 'DISGUST',
        'fear': 'FEAR',
        'happy': 'HAPPY',
        'neutral': 'NEUTRAL',
        'sad': 'SAD',
    }

    
    recognizable_emotion = emotion_mapping.get(predicted_emotion)
    return recognizable_emotion


In [ ]:
# import tensorflow as tf
# import librosa
# import numpy as np

# # Fungsi untuk memuat model dari file .h5
# def load_model(model_path):
#     model = tf.keras.models.load_model(model_path)
#     return model

# # Fungsi untuk memproses data audio dan melakukan prediksi
# def predict_audio_class(model, audio_path):
#     # Muat audio menggunakan librosa
#     audio_data, _ = librosa.load(audio_path, sr=44100)

#     # Lakukan ekstraksi fitur audio (misalnya, MFCC)
#     mfccs = librosa.feature.mfcc(y=audio_data, sr=44100, n_mfcc=13)

#     # Normalisasi data
#     mfccs = (mfccs - np.mean(mfccs)) / np.std(mfccs)

#     # Reshape data agar sesuai dengan input model
#     mfccs = mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1])

#     # Lakukan prediksi
#     predictions = model.predict(mfccs)

#     # Interpretasikan hasil prediksi (misalnya, ambil kelas dengan nilai tertinggi)
#     predicted_class = np.argmax(predictions)

#     return predicted_class

# # Path ke model .h5
# model_path = 'model_terbaik.h5'

# # Path ke file audio yang akan diprediksi
# audio_path = 'samples/sample 1.wav'

# # Memuat model
# model = load_model(model_path)

# # Melakukan prediksi
# predicted_class = predict_audio_class(model, audio_path)

# # Output hasil prediksi
# print(f'Hasil prediksi kelas audio: {predicted_class}')


In [ ]:

class EmotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Emotion Prediction")
        self.root.configure(bg='#2C3333')

        self.emotion_to_emoji = {
            "HAPPY": "emoji/Happy.jpg",
            "SAD": "emoji/Sad.jpg",
            "ANGRY": "emoji/Angry.jpg",
            "NEUTRAL": "emoji/Neutral.jpg",
            "FEAR": "emoji/Fear.jpg",
            "DISGUST": "emoji/Disgust.jpg"
        }

        self.emoji_image = None
        self.prediction_history = []

        self.show_home_page()

    def show_home_page(self):
        self.clear_window()

        label = tk.Label(self.root, text="Emotion Recognition",
                         font=('Poppins bold', 16))
        label.pack(pady=18)

        audio_button = tk.Button(self.root, text="Prediksi suara", font=(
            'Poppins', 8), command=self.show_audio_page,
            bg='#33691e', fg="#FBFBFB", width=14, height=1, pady=2)
        audio_button.pack(pady=0)

        history_button = tk.Button(self.root, text="Riwayat", font=(
            'Poppins', 8), command=self.show_history_page, bg='#283593', fg="#FBFBFB", width=14, height=1, pady=2)
        history_button.pack(pady=12)

        # about_button = tk.Button(self.root, text="About The App", command=self.show_about_page, bg='lightblue')
        # about_button.pack(pady=10)

    def show_audio_page(self):
        self.clear_window()

        canvas = tk.Canvas(self.root, width=500, height=500, bg='#FFFFFF')
        canvas.pack()

        title_label = tk.Label(self.root, text='Speech Recognition', font=(
            'Poppins bold', 26), bg="#FFFFFF", fg="#395B64")
        canvas.create_window(250, 50, window=title_label)

        def upload_audio():
            file_path = filedialog.askopenfilename(
                filetypes=[("Audio Files", "*.wav")])
            if file_path:
                predicted_emotion = predict_emotion(file_path)
                emotion_label.config(text=predicted_emotion)

                self.prediction_history.append(
                    (os.path.basename(file_path), predicted_emotion))

                emoji_image_path = self.emotion_to_emoji.get(predicted_emotion)
                if emoji_image_path:
                    emoji_image = Image.open(emoji_image_path)
                    emoji_image = emoji_image.resize(
                        (100, 100), Image.ANTIALIAS)
                    self.emoji_image = ImageTk.PhotoImage(emoji_image)
                    emoji_label.config(image=self.emoji_image)

        upload_button = tk.Button(self.root, text='Upload Audio', font=(
            'Poppins', 9), command=upload_audio, bg='#E7F6F2', fg="#395B64", width=14, height=1, pady=2)
        canvas.create_window(250, 150, window=upload_button)

        emotion_label = tk.Label(
            self.root, text='PREDICTED EMOTION WILL BE DISPLAY HERE', bg="#FFFFFF", font=('Poppins', 11))
        canvas.create_window(250, 200, window=emotion_label)

        emoji_label = tk.Label(self.root, image=None, bg="#FFFFFF")
        canvas.create_window(250, 300, window=emoji_label)

        back_button = tk.Button(
            self.root, text="Back to home", font=('Poppins', 9), command=self.show_home_page, bg='#E7F6F2', fg="#395B64", width=14, height=1, pady=2)
        canvas.create_window(250, 400, window=back_button)

    def show_history_page(self):
        self.clear_window()

        canvas = tk.Canvas(self.root, width=500, height=500, bg='#FFFFFF')
        canvas.pack()

        label = tk.Label(self.root, text="Prediction History",
                         font=('Poppins bold', 16), bg="#FFFFFF", fg="#395B64")
        canvas.create_window(250, 50, window=label)

        if self.prediction_history:
            for index, (file_name, predicted_emotion) in enumerate(self.prediction_history, start=1):
                history_text = f"{index}. File: {file_name}, Emotion: {predicted_emotion}"
                history_label = tk.Label(self.root, text=history_text, font=('Poppins', 10),bg="#FFFFFF", fg="#395B64")
                canvas.create_window(250, 100 + index * 30,
                                     window=history_label)
        else:
            no_history_label = tk.Label(
                self.root, text="No prediction history available.", font=('Poppins', 10),bg="#FFFFFF", fg="#B2533E")
            canvas.create_window(250, 150, window=no_history_label)

        back_button = tk.Button(
            self.root, text="Back to home", font=('Poppins', 9), command=self.show_home_page, bg='#E7F6F2', fg="#395B64", width=14, height=1, pady=2)
        canvas.create_window(250, 400, window=back_button)
    # def show_about_page(self):
    #     self.clear_window()

    #     canvas = tk.Canvas(self.root, width=500, height=500, bg='skyblue')
    #     canvas.pack()

    #     label = tk.Label(self.root, text="About The Software",
    #                      font=('Helvetica bold', 16))
    #     canvas.create_window(250, 50, window=label)

    #     about_text = ("Hello Everyone !! "
    #                   " Speech Emotion Recognition is a software that recognizes the emotion of the user."
    #                   " All of the audio files in this software should be inputted with '.wav' extension."
    #                   " A special thanks to the University of Toronto for the TESS data set and to all of my guiders"
    #                   " at clevered that guided me throughout the journey of making this software.")

    #     about_label = tk.Label(self.root, text=about_text, wraplength=400)
    #     canvas.create_window(250, 150, window=about_label)

    #     back_button = tk.Button(
    #         self.root, text="Back to Home", command=self.show_home_page)
    #     canvas.create_window(250, 400, window=back_button)

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    app = EmotionApp(root)
    root.mainloop()